In [ ]:
#! pip install t5
#import requests
#
#subject = input("What is your topic?")

#url = 'https://en.wikipedia.org/w/api.php'
#params={
#    'action':'query',
#    'format':'json',
#    'titles': subject,
#    'prop':'extracts',
#    'exintro':True,
#    'explaintext': True,
#}
#response = requests.get(url, params=params)
#data = response.json()
#
#page = next(iter(data['query']['pages'].values()))
#
#print(page['extract'][:])




In [ ]:
import csv
import requests
import os

def get_wikipedia_summary(topic):
    """Fetches the summary of a Wikipedia article for a given topic."""
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No summary available.")
    return None

def read_first_column(csv_filename):
    """Reads the first column from a CSV file and returns a list of topics."""
    topics = []
    with open(csv_filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:
                topics.append(row[0])
    return topics

def load_processed_topics(filename):
    """Loads previously processed topics from a file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(file.read().splitlines())
    return set()

def save_processed_topic(filename, topic):
    """Saves a topic to the processed topics file."""
    with open(filename, "a", encoding="utf-8") as file:
        file.write(topic + "\n")

def write_to_sasv(filename, data_list):
    """Writes a list of unique Wikipedia summaries to a .sasv file, separating entries with '/*'."""
    full_filename = f"{filename}.sasv"
    with open(full_filename, "a", encoding="utf-8") as file:
        file.write(" /* ".join(data_list) + " /* ")

def process_csv(csv_filename, sasv_filename, processed_filename):
    """Processes the first column of a CSV file, fetches Wikipedia summaries, and stores them uniquely."""
    topics = read_first_column(csv_filename)
    processed_topics = load_processed_topics(processed_filename)
    new_summaries = []
    
    for topic in topics:
        if topic not in processed_topics:
            summary = get_wikipedia_summary(topic)
            if summary:
                new_summaries.append(summary)
                save_processed_topic(processed_filename, topic)
    
    if new_summaries:
        write_to_sasv(sasv_filename, new_summaries)
        print(f"Stored {len(new_summaries)} new articles in {sasv_filename}.sasv")
    else:
        print("No new unique topics found.")

# Example usage
#process_csv("topics.csv", "articles", "processed_topics.txt")

